# Language modelling

The exercise shows how a language model may be used to solve word-prediction tasks and used to generate text.


## Tasks

1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.
>Done
2. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository.

In [1]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForMaskedLM, AutoModelWithLMHead
import torch

def import_model_tokenizer(name):
    model: AutoModel  = AutoModelWithLMHead.from_pretrained(name)
    tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(name)
    config: AutoConfig = AutoConfig.from_pretrained(name)
    return model, tokenizer

names = ["allegro/herbert-base-cased",
         "dkleczek/bert-base-polish-cased-v1",
        # "henryk/bert-base-multilingual-cased-finetuned-polish-squad1"
        "henryk/bert-base-multilingual-cased-finetuned-polish-squad2"
         # "DeepPavlov/bert-base-bg-cs-pl-ru-cased"
         ]


models_tokenizers = []
for name in names:
    model, tokenizer = import_model_tokenizer(name)
    models_tokenizers.append((model, tokenizer))

/home/micha/anaconda3/lib/python3.7/site-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceC

3. Produce the predictions for the following sentences (use each model and check 5 predictions):
   1. (M) Warszawa to największe `[MASK]`.
   1. (D) Te zabawki należą do `[MASK]`.
   1. (C) Policjant przygląda się `[MASK]`.
   1. (B) Na środku skrzyżowania widać `[MASK]`.
   1. (N) Właściciel samochodu widział złodzieja z `[MASK]`.
   1. (Ms) Prezydent z premierem rozmawiali wczoraj o `[MASK]`.
   1. (W) Witaj drogi `[MASK]`.

In [2]:
import re

def predict(sequence, model, tokenizer):
  sequence = re.sub(r"\?",tokenizer.mask_token, sequence)
  input = tokenizer.encode(sequence, return_tensors="pt")
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1][0].item()
  token_logits = model(input)[0]
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_5_tokens = torch.topk(mask_token_logits, 5)[-1]
  for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

def predict_for_examples(examples):
    for example in examples:
        for model_tokenizer in models_tokenizers:
            print(model_tokenizer[0].name_or_path)
            predict(example, model_tokenizer[0], model_tokenizer[1])

In [3]:
examples = [
            "Warszawa to największe ?.",
            "Te zabawki należą do ?.",
            "Policjant przygląda się ?.",
            "Na środku skrzyżowania widać ?.",
            "Właściciel samochodu widział złodzieja z ?.",
            "Prezydent z premierem rozmawiali wczoraj o ?.",
            "Witaj, drogi ?."
]

predict_for_examples(examples)

allegro/herbert-base-cased
Warszawa to największe miasto.
Warszawa to największe lotnisko.
Warszawa to największe centrum.
Warszawa to największe miasta.
Warszawa to największe atrakcje.
dkleczek/bert-base-polish-cased-v1
Warszawa to największe miasto.
Warszawa to największe województwo.
Warszawa to największe lotnisko.
Warszawa to największe miasteczko.
Warszawa to największe państwo.
henryk/bert-base-multilingual-cased-finetuned-polish-squad2
Warszawa to największe њ.
Warszawa to największe Kantoni.
Warszawa to największe hrvatskog.
Warszawa to największe ##czeństwa.
Warszawa to największe rež.
allegro/herbert-base-cased
Te zabawki należą do rodziny.
Te zabawki należą do nas.
Te zabawki należą do nich.
Te zabawki należą do najlepszych.
Te zabawki należą do ..
dkleczek/bert-base-polish-cased-v1
Te zabawki należą do ciebie.
Te zabawki należą do mnie.
Te zabawki należą do nas.
Te zabawki należą do pana.
Te zabawki należą do niego.
henryk/bert-base-multilingual-cased-finetuned-polish-squ

4. Check the model predictions for the following sentences (using each model):
   1. Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie `[MASK]`.
   1. Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie `[MASK]`.

In [4]:
examples = [
  "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ?.",
  "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie ?."
]

predict_for_examples(examples)

allegro/herbert-base-cased
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zdziwił.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie poddał.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zastanawiał.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie przyznał.
dkleczek/bert-base-polish-cased-v1
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgodził.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie martwił.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił.
henryk/bert-base-multilingual-cased-finetuned-polish-squad2
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##lució.
Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się

5. Check the model predictions for the following sentences:
   1. `[MASK]` wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
   1. W wakacje odwiedziłem `[MASK]`, który jest stolicą Islandii.
   1. Informatyka na `[MASK]` należy do najlepszych kierunków w Polsce.

In [5]:
examples = [
  "? wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.",
  "W wakacje odwiedziłem ?, który jest stolicą Islandii.",
  "Informatyka na ? należy do najlepszych kierunków w Polsce.",
]

predict_for_examples(examples)

allegro/herbert-base-cased
Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Ziemia wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Następnie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Ciało wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
dkleczek/bert-base-polish-cased-v1
Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Mięso wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
Piwo wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
henryk/bert-base-multilingual-cased-finetuned-polish-squad2
rež wrze w temperaturze 100 stopni, a zamarza w temeratu

6. Answer the following questions:
   1. Which of the models produced the best results?
   > Najlepsze rezultaty osiągnęły modele allegro/herbert-base-cased i dkleczek/bert-base-polish-cased-v1.

   1. Was any of the models able to capture Polish grammar?
   > Tak, modele allegro/herbert-base-cased oraz dkleczek/bert-base-polish-cased-v1 dobrze radzą sobie z polską gramatyką.
   > W zadaniu 3 te model w większości przypadków zastosowały odpowiednią rekcję.

   1. Was any of the models able to capture long-distant relationships between the words?
   > Tak, allegro/herbert-base-cased oraz dkleczek/bert-base-polish-cased-v1 uchwyciły długodystansową relację.
   > W zadaniu 4 te modele bezbłędnie wybrały właściwy rodzaj.

   1. Was any of the models able to capture world knowledge?
   > Modele allegro/herbert-base-cased i dkleczek/bert-base-polish-cased-v1 poradziły sobie dobrze z "testem wrzącej wody".
   > Niestety żaden z modeli nie potrafił właściwie wskazać stolicy Islandii - Rejkiawiku. Model allegro proponował miasta, jako jedyny dla wszystkich w top 5, jako stolicę.
   > Na podstawie tych wyników można powiedzieć, że modele uchwyciły (chociaż częściowo) wiedzę ogólną.

   1. What are the most striking errors made by the models?
   > Trzeci z wybranych modeli henryk/bert-base-multilingual-cased-finetuned-polish-squad2 (ale także modele henryk/bert-base-multilingual-cased-finetuned-polish-squad1 i DeepPavlov/bert-base-bg-cs-pl-ru-cased) prawdopodobnie ze względu na swoją wielojęzyczność dawały bardzo kiepskie wyniki.
   > Moją uwagę wśród wyników z dwóch najlepszych modeli zwróciły:
   > "to bym się nie zabił/zabiła"
   > "widział złodzieja z włamaniem"
   > "Warszawa to największe miasteczko"

## Hints

1. Language modelling (LM) is a task concentrated on computing the probability distribution of words given a sequence of
   preceding words.
2. In the past the most popular LM were based on n-gram counting. The distribution of probability of the next word was
   approximated by the relative frequency of the last n-words, preceding this word. Usually n=3, since larger values
   resulted in extremely large datasets.
3. Many algorithms were devised to improve that probability estimates for infrequent words. Among them Kneser-Ney was
   the most popular.
4. SRI LM is the most popular toolkit for creating traditional language models.
5. At present recurrent neural networks, attention networks and transformers are the most popular neural-network
   architectures for creating LMs.
6. The largest LM currently is GPT-3 described in (mind the number of authors!) *Language Models are Few-Shot Learners*
   Tom B. Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared Kaplan, Prafulla Dhariwal, Arvind Neelakantan, Pranav
   Shyam, Girish Sastry, Amanda Askell, Sandhini Agarwal, Ariel Herbert-Voss, Gretchen Krueger, Tom Henighan, Rewon
   Child, Aditya Ramesh, Daniel M. Ziegler, Jeffrey Wu, Clemens Winter, Christopher Hesse, Mark Chen, Eric Sigler,
   Mateusz Litwin, Scott Gray, Benjamin Chess, Jack Clark, Christopher Berner, Sam McCandlish, Alec Radford, Ilya
   Sutskever, Dario Amodei